In [1]:
from google.colab import drive
drive.mount('./content')

Mounted at ./content


In [ ]:
import keras
keras.__version__

'2.4.3'

In [ ]:
from keras import layers
import numpy as np

latent_dim = 32
height = 32
width = 32
channels = 3

generator_input = keras.Input(shape=(latent_dim,))

# 입력을 16 × 16 크기의 128개 채널을 가진 특성 맵으로 변환합니다
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

# 합성곱 층을 추가합니다
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# 32 × 32 크기로 업샘플링합니다
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

# 합성곱 층을 더 추가합니다
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# 32 × 32 크기의 1개 채널을 가진 특성 맵을 생성합니다
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32768)             0         
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 256)       104883

In [ ]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

# 드롭아웃 층을 넣는 것이 아주 중요합니다!
x = layers.Dropout(0.4)(x)

# 분류 층
x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

# 옵티마이저에서 (값을 지정하여) 그래디언트 클리핑을 사용합니다
# 안정된 훈련을 위해서 학습률 감쇠를 사용합니다
discriminator_optimizer = keras.optimizers.RMSprop(lr=0.0008, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 6, 6, 128)         0   

In [ ]:
# 판별자의 가중치가 훈련되지 않도록 설정합니다(gan 모델에만 적용됩니다)
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

In [ ]:
import os
from keras.preprocessing import image

In [ ]:
img_dir = '/content/content/MyDrive/Colab Notebooks/sai/10img.zip (Unzipped Files)'

In [ ]:
from PIL import Image
from numpy import asarray
x_train = []

In [ ]:
for i in os.listdir(img_dir):
  # load the image
  image = Image.open(os.path.join(img_dir, i))
  # convert image to numpy array
  data = asarray(image)
  x_train.append(data)
x_train= asarray(x_train)
x_train = x_train/255.

In [ ]:
import keras
keras.__version__

'2.4.3'

In [ ]:
x_train= asarray(x_train)

In [ ]:
x_train.shape

(504, 32, 32, 3)

In [ ]:
# 데이터를 정규화합니다
x_train = x_train.reshape(
    (x_train.shape[0],) + (height, width, channels)).astype('float32') / 255.

In [ ]:
iterations = 100
batch_size = 64
save_dir = './gan_images/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

# 훈련 반복 시작
start = 0
for step in range(iterations):
    # 잠재 공간에서 무작위로 포인트를 샘플링합니다
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # 가짜 이미지를 디코딩합니다
    generated_images = generator.predict(random_latent_vectors)

    # 진짜 이미지와 연결합니다
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])

    # 진짜와 가짜 이미지를 구분하여 레이블을 합칩니다
    labels = np.concatenate([np.ones((batch_size, 1)),
                             np.zeros((batch_size, 1))])
    # 레이블에 랜덤 노이즈를 추가합니다. 아주 중요합니다!
    labels += 0.05 * np.random.random(labels.shape)

    # discriminator를 훈련합니다
    d_loss = discriminator.train_on_batch(combined_images, labels)

    # 잠재 공간에서 무작위로 포인트를 샘플링합니다
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # 모두 “진짜 이미지"라고 레이블을 만듭니다
    misleading_targets = np.zeros((batch_size, 1))

    # generator를 훈련합니다(gan 모델에서 discriminator의 가중치는 동결됩니다)
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
      start = 0

    # 중간 중간 저장하고 그래프를 그립니다
    if step % 10 == 0:
        # 모델 가중치를 저장합니다
        gan.save_weights('gan.h5')

        # 측정 지표를 출력합니다
        print('스텝 %s에서 판별자 손실: %s' % (step, d_loss))
        print('스텝 %s에서 적대적 손실: %s' % (step, a_loss))

        # 생성된 이미지 하나를 저장합니다
        # img = image.array_to_img(generated_images[0] * 255., scale=False)
        # img = generated_images[0] * 255.
        img = Image.fromarray(generated_images[0] * 255., 'RGB')
        img.save(os.path.join(save_dir, 'generated_img' + str(step) + '.png'))

        # 비교를 위해 진짜 이미지 하나를 저장합니다
        # img = image.array_to_img(real_images[0] * 255., scale=False)
        # img = real_images[0] * 255.
        img = Image.fromarray(real_images[0] * 255., 'RGB')
        img.save(os.path.join(save_dir, 'real_img' + str(step) + '.png'))

스텝 0에서 판별자 손실: 0.6918667554855347
스텝 0에서 적대적 손실: 0.66827392578125
스텝 10에서 판별자 손실: 0.6819419860839844
스텝 10에서 적대적 손실: 0.8649263381958008
스텝 20에서 판별자 손실: 0.5943122506141663
스텝 20에서 적대적 손실: 4.014800071716309
스텝 30에서 판별자 손실: 0.7129910588264465
스텝 30에서 적대적 손실: 0.603651762008667
스텝 40에서 판별자 손실: 0.6862527132034302
스텝 40에서 적대적 손실: 0.7677250504493713
스텝 50에서 판별자 손실: 0.7043185830116272
스텝 50에서 적대적 손실: 0.8451831340789795
스텝 60에서 판별자 손실: 0.6871731281280518
스텝 60에서 적대적 손실: 0.815424382686615
스텝 70에서 판별자 손실: 0.6809022426605225
스텝 70에서 적대적 손실: 0.8036088943481445
스텝 80에서 판별자 손실: 0.725641667842865
스텝 80에서 적대적 손실: 1.0499374866485596
스텝 90에서 판별자 손실: 0.6690715551376343
스텝 90에서 적대적 손실: 1.0151737928390503


In [ ]:
import time

iterations = 1000
batch_size = 64
save_dir = './gan_images/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

# 훈련 반복 시작
start = 0
for step in range(iterations):
    s = time.time()
    # 잠재 공간에서 무작위로 포인트를 샘플링합니다
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # 가짜 이미지를 디코딩합니다
    generated_images = generator.predict(random_latent_vectors)

    # 진짜 이미지와 연결합니다
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])

    # 진짜와 가짜 이미지를 구분하여 레이블을 합칩니다
    labels = np.concatenate([np.ones((batch_size, 1)),
                             np.zeros((batch_size, 1))])
    # 레이블에 랜덤 노이즈를 추가합니다. 아주 중요합니다!
    labels += 0.05 * np.random.random(labels.shape)

    # discriminator를 훈련합니다
    d_loss = discriminator.train_on_batch(combined_images, labels)

    # 잠재 공간에서 무작위로 포인트를 샘플링합니다
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # 모두 “진짜 이미지"라고 레이블을 만듭니다
    misleading_targets = np.zeros((batch_size, 1))

    # generator를 훈련합니다(gan 모델에서 discriminator의 가중치는 동결됩니다)
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
      start = 0

    # 중간 중간 저장하고 그래프를 그립니다
    if step % 100 == 0:
        # 모델 가중치를 저장합니다
        gan.save_weights('gan.h5')

        # 측정 지표를 출력합니다
        print('스텝 %s에서 판별자 손실: %s' % (step, d_loss))
        print('스텝 %s에서 적대적 손실: %s' % (step, a_loss))

        # 생성된 이미지 하나를 저장합니다
        # img = image.array_to_img(generated_images[0] * 255., scale=False)
        # img = generated_images[0] * 255.
        img = Image.fromarray(generated_images[0] * 255., 'RGB')
        img.save(os.path.join(save_dir, 'generated_img' + str(step) + '.png'))

        # 비교를 위해 진짜 이미지 하나를 저장합니다
        # img = image.array_to_img(real_images[0] * 255., scale=False)
        # img = real_images[0] * 255.
        img = Image.fromarray(real_images[0] * 255., 'RGB')
        img.save(os.path.join(save_dir, 'real_img' + str(step) + '.png'))
    e = time.time()
    print('%d번째 소요시간: ' %step ,end='')
    print(e-s)

스텝 0에서 판별자 손실: 0.6842163801193237
스텝 0에서 적대적 손실: 0.7664318084716797
0번째 소요시간: 37.90692615509033
1번째 소요시간: 38.30387353897095
2번째 소요시간: 38.31021213531494
3번째 소요시간: 38.031718492507935
4번째 소요시간: 38.531421422958374
5번째 소요시간: 38.028552293777466
6번째 소요시간: 37.97904992103577
7번째 소요시간: 38.01354765892029
8번째 소요시간: 38.176384687423706
9번째 소요시간: 37.90229320526123
10번째 소요시간: 37.841352701187134
11번째 소요시간: 37.7318069934845
12번째 소요시간: 37.81803345680237
13번째 소요시간: 37.48621678352356
14번째 소요시간: 37.444746017456055
15번째 소요시간: 37.50986123085022
16번째 소요시간: 37.63456654548645
17번째 소요시간: 38.83602237701416
18번째 소요시간: 38.14484691619873
19번째 소요시간: 38.01335287094116
20번째 소요시간: 38.056427240371704
21번째 소요시간: 37.483389377593994
22번째 소요시간: 37.47182583808899
23번째 소요시간: 37.440905809402466
24번째 소요시간: 37.58780860900879
25번째 소요시간: 37.661043643951416
26번째 소요시간: 37.60299372673035
27번째 소요시간: 37.49180030822754
28번째 소요시간: 38.02420163154602
29번째 소요시간: 38.16111731529236
30번째 소요시간: 37.868892431259155
31번째 소요시간: 37.78772163391113
32번째

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# 잠재 공간에서 랜덤한 포인트를 샘플링합니다
random_latent_vectors = np.random.normal(size=(10, latent_dim))

# 가짜 이미지로 디코딩합니다
generated_images = generator.predict(random_latent_vectors)

for i in range(generated_images.shape[0]):
    img = image.array_to_img(generated_images[i] * 255., scale=False)
    plt.figure()
    plt.imshow(img)
    
plt.show()